<a href="https://colab.research.google.com/github/Sheraz55/Deep-learning-course-practice/blob/main/Neaural_transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
from keras.preprocessing.image import load_img, img_to_array
target_image_path = '/content/Sheraz_pic (2).jpg'
style_reference_image_path = '/content/style.jpg'
width, height = load_img(target_image_path).size
img_height = 400
img_width = int(width * img_height / height)

In [36]:
import numpy as np
from keras.applications import vgg19
def preprocess_image(image_path):
  img = load_img(image_path, target_size=(img_height, img_width))
  img = img_to_array(img)
  img = np.expand_dims(img, axis=0)
  img = vgg19.preprocess_input(img)
  return img

In [37]:
def deprocess_image(x):
  x[:, :, 0] += 103.939
  x[:, :, 1] += 116.779
  x[:, :, 2] += 123.68
  x = x[:, :, ::-1]
  x = np.clip(x, 0, 255).astype('uint8')
  return x

In [38]:
from keras import backend as K
target_image = K.constant(preprocess_image(target_image_path))
style_reference_image = K.constant(preprocess_image(style_reference_image_path))
combination_image = K.placeholder((1, img_height, img_width, 3))

input_tensor = K.concatenate([target_image,style_reference_image,combination_image], axis=0)
model = vgg19.VGG19(input_tensor=input_tensor,weights='imagenet',include_top=False)
print('Model loaded.')

Model loaded.


In [39]:
def content_loss(base, combination):
  return K.sum(K.square(combination - base))

In [40]:
def gram_matrix(x):
  features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
  gram = K.dot(features, K.transpose(features))
  return gram
def style_loss(style, combination):
  S = gram_matrix(style)
  C = gram_matrix(combination)
  channels = 3
  size = img_height * img_width
  return K.sum(K.square(S - C)) / (4. * (channels ** 2) * (size ** 2))

In [41]:
def total_variation_loss(x):
  a = K.square(
    x[:, :img_height - 1, :img_width - 1, :] -
    x[:, 1:, :img_width - 1, :])
  b = K.square(
    x[:, :img_height - 1, :img_width - 1, :] -
    x[:, :img_height - 1, 1:, :])
  return K.sum(K.pow(a + b, 1.25))

In [42]:
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])
content_layer = 'block5_conv2'
style_layers = ['block1_conv1',
                'block2_conv1',
                'block3_conv1',
                'block4_conv1',
                'block5_conv1']
total_variation_weight = 1e-4
style_weight = 1.
content_weight = 0.025

In [43]:

loss = K.variable(0.)
layer_features = outputs_dict[content_layer]
target_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]



In [44]:
loss = loss + (content_weight * content_loss(target_image_features,combination_features))



In [45]:
for layer_name in style_layers:
  layer_features = outputs_dict[layer_name]
  style_reference_features = layer_features[1, :, :, :]
  combination_features = layer_features[2, :, :, :]
  sl = style_loss(style_reference_features, combination_features)
  loss = loss + ( (style_weight / len(style_layers)) * sl)  
loss = loss + ( total_variation_weight * total_variation_loss(combination_image))

In [46]:
import tensorflow as tf

In [47]:
from tensorflow.python.framework.ops import disable_eager_execution
disable_eager_execution()
tf.compat.v1.disable_eager_execution()
grads = K.gradients(loss, combination_image)[0]
fetch_loss_and_grads = K.function([combination_image], [loss, grads])

class Evaluator(object):
  def __init__(self):
    self.loss_value = None
    self.grads_values = None
  def loss(self, x):
    assert self.loss_value is None
    x = x.reshape((1, img_height, img_width, 3))
    outs = fetch_loss_and_grads([x])
    loss_value = outs[0]
    grad_values = outs[1].flatten().astype('float64')
    self.loss_value = loss_value
    self.grad_values = grad_values
    return self.loss_value
  def grads(self, x):
    assert self.loss_value is not None
    grad_values = np.copy(self.grad_values)
    self.loss_value = None
    self.grad_values = None
    return grad_values
evaluator = Evaluator()

In [48]:
from scipy.optimize import fmin_l_bfgs_b
from keras.preprocessing.image import save_img

import time

result_prefix = 'my_result'
iterations = 20

x = preprocess_image(target_image_path)
x = x.flatten()


In [49]:
for i in range(iterations):
  print('Start of iteration', i)
  start_time = time.time()
  x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x, fprime=evaluator.grads, maxfun=20)
  print('Current loss value:', min_val)


Start of iteration 0
Current loss value: 3479402500.0
Start of iteration 1
Current loss value: 1819736000.0
Start of iteration 2
Current loss value: 1223957900.0
Start of iteration 3
Current loss value: 908784200.0
Start of iteration 4
Current loss value: 713086800.0
Start of iteration 5
Current loss value: 582444860.0
Start of iteration 6
Current loss value: 506739650.0
Start of iteration 7
Current loss value: 459776700.0
Start of iteration 8
Current loss value: 418226850.0
Start of iteration 9
Current loss value: 385381700.0
Start of iteration 10
Current loss value: 363281440.0
Start of iteration 11
Current loss value: 342898880.0
Start of iteration 12
Current loss value: 328140100.0
Start of iteration 13
Current loss value: 311749220.0
Start of iteration 14
Current loss value: 299554660.0
Start of iteration 15
Current loss value: 289515550.0
Start of iteration 16
Current loss value: 280505600.0
Start of iteration 17
Current loss value: 272213440.0
Start of iteration 18
Current loss 

In [56]:

img = x.copy().reshape((img_height, img_width, 3))
img = deprocess_image(img)
fname = result_prefix + '_at_iteration_%d.png' % i
save_img(fname, img)
print('Image saved as', fname)
end_time = time.time()
print('Iteration %d completed in %ds' % (i, end_time - start_time))

Image saved as my_result_at_iteration_19.png
Iteration 19 completed in 554s


In [57]:
img = x.copy().reshape((img_height, img_width, 3))
img = deprocess_image(img)
fname = result_prefix + '_at_iteration_%d.png' % i
save_img(fname, img)
print('Image saved as', fname)
end_time = time.time()
print('Iteration %d completed in %ds' % (i, end_time - start_time))

Image saved as my_result_at_iteration_19.png
Iteration 19 completed in 603s
